In [27]:
#all imports
import json
import pandas as pd
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
from sklearn.cluster import KMeans
import spacy
import re

In [22]:
data=pd.read_json(r'News_Category_Dataset_v2.json',lines=True)

In [23]:
environment=(data["category"]=="ENVIRONMENT")

In [24]:
dataset=data[environment]
dataset=dataset["headline"]

In [43]:
stemmer = SnowballStemmer('english')
tokenizer = RegexpTokenizer(r'[a-zA-Z\']+')
#based on part of speech, the tokenize function would lemmatize the word
spacy_obj=spacy.load('en',disable=['parser', 'ner'])
def tokenize(text):
    text=re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", text)
    text = re.sub("\d+", "", text)
    lemmatized=spacy_obj(text.lower())
    return [token.lemma_ for token in lemmatized]


In [56]:
punc = ['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']',"//",".org","'s", '{', '}',"%","$","&","+","=","-","--","-PRON-","-pron-","..","...","/"]
stop_words = text.ENGLISH_STOP_WORDS.union(punc)
vectorizer = TfidfVectorizer(stop_words = stop_words,tokenizer=tokenize,max_features=2000)
vectorized=vectorizer.fit_transform(dataset)

/home/jayaram/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['make'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [57]:
words=vectorizer.get_feature_names()
kmeans = KMeans(n_clusters = 23, n_init = 10, n_jobs = -1)
kmeans.fit(vectorized)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=23, n_init=10, n_jobs=-1, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [58]:
common_words = kmeans.cluster_centers_.argsort()[:,-1:-11:-1]
for num, centroid in enumerate(common_words):
    print(str(num) + ' : ' + ', '.join(words[word] for word in centroid))

0 : sound, metal, backyard, recording, hear, music, audio, cicada, scrap, temperature
1 : climate, change, talk, week, need, say, study, ice, action, carbon
2 : new, specie, york, farmer, discover, photo, energy, normal, animal, nyc
3 : sea, rise, level, video, study, fall, photo, underwater, deep, melting
4 : day, earth, mother, world, gift, photo, fall, green, celebrate, endanger
5 : baby, photo, animal, week, panda, picture, zoo, elephant, gorilla, giraffes
6 : animal, week, photo, picture, tiger, elephant, monkey, panda, lion, cub
7 :  , use, park, photo, national, amazing, animal, odd, weather.com, way
8 : time, celebrate, lapse, year, video, week, cool, wildlife, puppy, draw
9 : keystone, xl, pipeline, sand, tar, obama, protester, arrest, student, department
10 : extreme, weather, week, photo, hero, hill, hilarious, highlight, high, zoonose
11 : photo, animal, life, nature, thing, pollution, world, big, storm, free
12 : spill, bp, oil, trial, gulf, exxon, molasse, testify, hallib

cluster 17,cluster 3,cluster 12,cluster 21,cluster 22, cluster 20. relevant water words and some of the irrelevant words with some connection to environment.